# Module 2

In [226]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)
import edhec_risk_kit as erk

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Question 1
Use the EDHEC Hedge Fund Indices data set that we used in the lab assignment as well as in the previous week’s assignments. Load them into Python and perform the following analysis based on data since 2000 (including all of 2000): What was the Monthly Parametric Gaussian VaR at the 1% level (as a +ve number) of the Distressed Securities strategy?

Enter the positive number as a percent .e.g. For 5.32% enter 5.32

In [227]:
ind = erk.get_ind_returns()
er = erk.annualize_rets(ind["2000":], 12)
cov = ind["2000":].cov()

In [228]:
hfi = erk.get_hfi_returns()

In [229]:
hfi_2000 = hfi["2000":]

In [230]:
list = [erk.var_gaussian(hfi_2000, level=1), erk.var_gaussian(hfi_2000, level=1, modified=True), erk.var_historic(hfi_2000, level=1)]
comparison = pd.concat(list, axis=1)
comparison.columns = ['Gaussian', 'Cornish-Fisher', 'Historic']
comparison.loc["Distressed Securities"]

Gaussian          0.031430
Cornish-Fisher    0.049669
Historic          0.042642
Name: Distressed Securities, dtype: float64

In [231]:
list = [erk.var_gaussian(hfi_2000, level=5), erk.var_gaussian(hfi_2000, level=5, modified=True), erk.var_historic(hfi_2000)]
comparison = pd.concat(list, axis=1)
comparison.columns = ['Gaussian', 'Cornish-Fisher', 'Historic']
comparison.loc["Distressed Securities"]

Gaussian          0.020280
Cornish-Fisher    0.023455
Historic          0.019580
Name: Distressed Securities, dtype: float64

## Question 4
Next, load the 30 industry return data using the erk.get_ind_returns() function that we developed during the lab sessions. For purposes of the remaining questions, use data during the 5 year period 2013-2017 (both inclusive) to estimate the expected returns as well as the covariance matrix. To be able to respond to the questions, you will need to build the MSR, EW and GMV portfolios consisting of the “Books”, “Steel”, "Oil", and "Mines" industries. Assume the risk free rate over the 5 year period is 10%.

In [232]:
ind = erk.get_ind_returns()
ind_5 = ind["2013":"2017"]
riskfree_rate = 0.1

portfolios_list = ["Books", "Steel", "Oil", "Mines"]
ind_5 = ind_5[portfolios_list]

er = erk.annualize_rets(ind_5, 12)
cov = ind_5.cov()
#cov = cov.loc[portfolios_list, portfolios_list]

float_formatter = "{:.20f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

In [233]:
er

Books    0.095312
Steel    0.100716
Oil      0.025275
Mines    0.005136
dtype: float64

### MSR

In [234]:
w_msr = erk.msr(riskfree_rate, er, cov)
w_msr

array([0.00000000000000000000, 1.00000000000000000000,
       0.00000000000000002082, 0.00000000000000060195])

### GMV

In [235]:
w_gmv = erk.gmv(cov)
w_gmv

array([0.47702487014389954867, 0.00000000000000000000,
       0.43411202519772174035, 0.08886310465837875261])

11. Assume two different investors invested in the GMV and MSR portfolios at the start of 2018 using the weights we just computed. Compute the annualized volatility of these two portfolios over the next 12 months of 2018? (Hint: Use the portfolio_vol code we developed in the lab and use ind\[“2018”\]\[l\].cov() to compute the covariance matrix for 2018, assuming that the variable ind holds the industry returns and the variable l holds the list of industry portfolios you are willing to hold. Don’t forget to annualize the volatility)

**What would be the annualized volatility over 2018 using the weights of the MSR portfolio?**

In [236]:
ind_2018 = ind["2018"][portfolios_list]
ind_2018

,Books,Steel,Oil,Mines
2018-01,0.0449,0.0361,0.0286,0.0357
2018-02,-0.0325,-0.0199,-0.1102,-0.0335
2018-03,0.0031,-0.0586,0.0308,-0.0015
2018-04,-0.0062,0.0366,0.1030,-0.0339
2018-05,-0.0061,0.0562,0.0343,0.0308
2018-06,0.0218,-0.0514,0.0052,-0.0178
2018-07,-0.0417,0.0424,0.0104,-0.0224
2018-08,-0.0447,-0.0323,-0.0295,-0.1075
2018-09,-0.0221,-0.0067,0.0356,-0.0173
2018-10,-0.0666,-0.1161,-0.1206,-0.1032


In [237]:
cov_2018 = ind_2018.cov()
cov_2018

,Books,Steel,Oil,Mines
Books,0.002799,0.002098,0.002558,0.001629
Steel,0.002098,0.004027,0.003473,0.001843
Oil,0.002558,0.003473,0.005296,0.001961
Mines,0.001629,0.001843,0.001961,0.002059


In [238]:
vol_msr = erk.portfolio_vol(w_msr, cov_2018)
vol_msr

0.06346230747220724

To annualize volatility we have to multiply by square root of 12

In [239]:
vol_msr * (12**0.5)

0.21983988181484188

12. What would be the annualized volatility over 2018 using the weights of the GMV portfolio? (Reminder and Hint: Use the portfolio_vol code we developed in the lab and use ind\[“2018”\]\[l\].cov() to compute the covariance matrix for 2018, assuming that the variable ind holds the industry returns and the variable l holds the list of industry portfolios you are willing to hold. Don’t forget to annualize the volatility)

### GMV

In [240]:
vol_gmv = erk.portfolio_vol(w_gmv, cov_2018)
vol_gmv

0.05477245366380528

In [241]:
vol_gmv * (12**0.5)

0.18973734520184568

# Summary

Volatility of GMV portfolio is lower.